In [48]:
!pip install pyngrok streamlit transformers torch streamlit-chat

In [50]:
%%writefile app.py
import streamlit as st
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from datetime import datetime

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can switch to other smaller models like "distilgpt2" for faster response
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Set up the Streamlit UI
st.set_page_config(page_title="Free Coding Assistant Chatbot", page_icon="🤖", layout="wide")

# Title of the app
st.title("💻 Free Coding Assistant Chatbot")

# Initialize session state for conversation history if it doesn't exist
if "conversation_history" not in st.session_state:
    st.session_state.conversation_history = []

# Function to generate responses from GPT-2
def generate_response(user_input, conversation_history):
    # Append the user input to the conversation history
    conversation_history.append(f"User: {user_input}")

    # Create the input for GPT-2 by concatenating the conversation history
    input_text = "\n".join(conversation_history)

    # Encode the input text and check for input length
    inputs = tokenizer.encode(input_text, return_tensors="pt")

    # Truncate the input if it exceeds the max input length of GPT-2 (1024 tokens)
    max_input_length = 1024  # GPT-2's token limit
    if inputs.size(1) > max_input_length:
        inputs = inputs[:, -max_input_length:]

    # Generate a response using GPT-2 with max_new_tokens instead of max_length
    outputs = model.generate(
        inputs,
        max_new_tokens=150,  # Limit the number of new tokens to generate (e.g., 150 tokens)
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return the response
    bot_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # The GPT model sometimes includes the entire conversation, so let's clean it up
    bot_response = bot_response.replace(input_text, "").strip()

    # Add bot response to conversation history
    conversation_history.append(f"Bot: {bot_response}")

    return bot_response, conversation_history

# User input section with placeholder
user_input = st.text_input("Ask me anything about coding:", placeholder="Type your coding question here...")

# User selects the type of interaction
interaction_type = st.selectbox("Choose Interaction Type:", ["Code Explanation", "Algorithm Suggestion", "Bug Fixing", "General Query"])

# Set response length control
response_length = st.slider("Select Response Length:", 50, 300, 150)

# Show loading spinner while waiting for response
if user_input:
    with st.spinner("Generating response..."):
        # Get the response from GPT-2 model
        bot_response, st.session_state.conversation_history = generate_response(user_input, st.session_state.conversation_history)

    # Display the chatbot response
    st.markdown(f"### **Bot:** {bot_response}")

# Add a button to clear the chat history
if st.button('Clear Chat History'):
    st.session_state.conversation_history = []
    st.success("Chat history cleared!")

# Option to save conversation as a text file
if st.button('Save Conversation'):
    conversation_text = "\n".join(st.session_state.conversation_history)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"conversation_{timestamp}.txt"
    with open(file_name, 'w') as file:
        file.write(conversation_text)
    st.success(f"Conversation saved as {file_name}")

# Instructions or information with a styled section
st.markdown("""
    #### Instructions:
    1. Ask coding-related questions such as: "What is a Python decorator?" or "How do I implement a binary search?"
    2. Choose an interaction type for more specific responses (Code Explanation, Algorithm Suggestion, Bug Fixing, etc.)
    3. Adjust the response length using the slider for shorter or longer answers.
    4. The chatbot will respond with code explanations, code snippets, or other helpful information.
    5. Type your question and press **Enter** to get an answer.
    6. Use the **Clear Chat History** button to start a fresh conversation.
    7. You can also **Save the conversation** for future reference.
""")

# Footer
st.markdown("---")
st.markdown("Made with 💙 by ChatGPT | Powered by GPT-2 (Free)")

# Display conversation history with timestamps
if st.session_state.conversation_history:
    st.markdown("### Conversation History:")
    for message in st.session_state.conversation_history:
        # Add a timestamp to each message
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        role, text = message.split(":", 1)
        st.markdown(f"**[{timestamp}] {role}:** {text.strip()}")


Overwriting app.py


<NgrokTunnel: "https://f1fb-34-31-236-182.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.31.236.182:8501

